In [1]:
import openmeteo_requests

In [2]:
import requests_cache
import pandas as pd
from retry_requests import retry

In [3]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [4]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive?latitude=25&longitude=45&start_date=2021-01-01&end_date=2024-01-01&daily=weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_mean,sunrise,sunset,daylight_duration,rain_sum,precipitation_hours,wind_speed_10m_max,wind_direction_10m_dominant&timezone=auto"
params = {
	"latitude": 25,
	"longitude": 45,
	"start_date": "2023-01-01",
	"end_date": "2024-01-01",
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_mean", "sunrise", "sunset", "daylight_duration", "rain_sum", "precipitation_hours", "wind_speed_10m_max", "wind_direction_10m_dominant"],
	"timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)

In [5]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 24.9912109375°N 45.048439025878906°E
Elevation 789.0 m asl
Timezone b'Asia/Riyadh' b'+03'
Timezone difference to GMT+0 10800 s


In [6]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_mean = daily.Variables(4).ValuesAsNumpy()
daily_sunrise = daily.Variables(5).ValuesAsNumpy()
daily_sunset = daily.Variables(6).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(7).ValuesAsNumpy()
daily_rain_sum = daily.Variables(8).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(9).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(10).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(11).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s"),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["rain_sum"] = daily_rain_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)


                    date  weather_code  temperature_2m_max  \
0    2020-12-31 21:00:00           1.0           20.274000   
1    2021-01-01 21:00:00           1.0           21.274000   
2    2021-01-02 21:00:00           1.0           22.824001   
3    2021-01-03 21:00:00          51.0           25.474001   
4    2021-01-04 21:00:00           1.0           24.174002   
...                  ...           ...                 ...   
1091 2023-12-27 21:00:00           2.0           25.424002   
1092 2023-12-28 21:00:00          63.0           21.474001   
1093 2023-12-29 21:00:00          61.0           22.574001   
1094 2023-12-30 21:00:00          51.0           21.024000   
1095 2023-12-31 21:00:00           3.0           19.774000   

      temperature_2m_min  temperature_2m_mean  apparent_temperature_mean  \
0                 11.824            15.705253                  12.249824   
1                 10.924            15.905251                  13.669918   
2                 10.924   